In [3]:
import pandas as pd

In [4]:
from glob import glob
parquetlist = glob('../input/otto-chunk-data-inparquet-format/test_parquet/*.parquet')

In [5]:
parquetlist

['../input/otto-chunk-data-inparquet-format/test_parquet/000500000_000600000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/000200000_000300000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/000100000_000200000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/001500000_001600000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/000400000_000500000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/000800000_000900000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/001200000_001300000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/001300000_001400000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/000900000_001000000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/001000000_001100000.parquet',
 '../input/otto-chunk-data-inparquet-format/test_parquet/001600000_001700000.parquet',
 '../input/otto-chunk-data-inparquet-format

In [6]:
from tqdm import tqdm
dataall = []
for xx in tqdm(range(len(parquetlist))):
    data = pd.read_parquet(parquetlist[xx])
    dataall.append(data)
    
dataall = pd.concat(dataall).reset_index(drop=True)

100%|██████████| 17/17 [00:00<00:00, 22.06it/s]


In [7]:
dataall

,session,aid,ts,type
0,13399779,110716,1661885147895,clicks
1,13399779,738837,1661885164597,clicks
2,13399780,557072,1661885147958,clicks
3,13399780,770350,1661885172583,clicks
4,13399781,1062698,1661885147979,clicks
...,...,...,...,...
6928118,13599775,232605,1661959824002,clicks
6928119,13599776,812507,1661958067254,clicks
6928120,13599776,587309,1661958145310,clicks
6928121,13599777,1583223,1661958067311,clicks


In [8]:
parquetlist = glob('../input/otto-chunk-data-inparquet-format/train_parquet/*.parquet')

In [9]:
from tqdm import tqdm
traindataall = []
for xx in tqdm(range(len(parquetlist))):
    data = pd.read_parquet(parquetlist[xx])
    traindataall.append(data)
    
traindataall = pd.concat(traindataall).reset_index(drop=True)

100%|██████████| 129/129 [00:21<00:00,  6.04it/s]


In [10]:
traindataall

,session,aid,ts,type
0,7600000,45494,1660352198353,clicks
1,7600000,214334,1660352290058,clicks
2,7600000,269405,1660352332852,clicks
3,7600000,269405,1660352380296,clicks
4,7600000,269405,1660469933882,clicks
...,...,...,...,...
216716091,12699998,812200,1661681939298,clicks
216716092,12699999,756588,1661681845934,clicks
216716093,12699999,1474940,1661681937448,clicks
216716094,12699999,490171,1661682033374,clicks


In [11]:
# traindataall[traindataall.session==11098534].sort_values(by='ts')

In [22]:
import time
from datetime import datetime
time_local = datetime.fromtimestamp(dataall.ts.min()/1000)
time_local

datetime.datetime(2022, 8, 28, 22, 0, 0, 278000)

In [23]:
import time
from datetime import datetime
time_local = datetime.fromtimestamp(dataall.ts.max()/1000)
time_local

datetime.datetime(2022, 9, 4, 21, 59, 51, 563000)

In [24]:
import time
from datetime import datetime
time_local = datetime.fromtimestamp(traindataall.ts.min()/1000)
time_local

datetime.datetime(2022, 7, 31, 22, 0, 0, 25000)

In [25]:
import time
from datetime import datetime
time_local = datetime.fromtimestamp(traindataall.ts.max()/1000)
time_local

datetime.datetime(2022, 8, 28, 21, 59, 59, 984000)

In [26]:
from datetime import timedelta
time_elapsed = dataall['ts'].max()-dataall.ts.min()
str(timedelta(milliseconds=float(time_elapsed)))

'6 days, 23:59:51.285000'

In [27]:
from datetime import timedelta
time_elapsed = traindataall['ts'].max()-traindataall.ts.min()
str(timedelta(milliseconds=float(time_elapsed)))

'27 days, 23:59:59.959000'

In [18]:
time_elapsed

604791285

In [15]:
dataall.ts.min()

1661724000278

In [17]:
dataall

,session,aid,ts,type
0,13399779,110716,1661885147895,clicks
1,13399779,738837,1661885164597,clicks
2,13399780,557072,1661885147958,clicks
3,13399780,770350,1661885172583,clicks
4,13399781,1062698,1661885147979,clicks
...,...,...,...,...
6928118,13599775,232605,1661959824002,clicks
6928119,13599776,812507,1661958067254,clicks
6928120,13599776,587309,1661958145310,clicks
6928121,13599777,1583223,1661958067311,clicks


In [19]:
type_weight_multipliers = {'clicks': 1, 'carts': 6, 'orders': 3}


In [20]:
dataall0 = dataall.copy()
dataall0['type'] = dataall0['type'].apply(lambda x:type_weight_multipliers[x])
user_features0 = dataall0.groupby('session').agg({'session':'count','aid':'nunique','type':'mean'})
user_features0.columns = ['user_count','user_item_count','user_buy_ratio']

user_features1 = dataall[dataall.type=='clicks'].groupby('session').agg({'session':'count','aid':'nunique'})
user_features1.columns = ['user_clicks_count','user_clicks_item_count']

user_features2 = dataall[dataall.type=='carts'].groupby('session').agg({'session':'count','aid':'nunique'})
user_features2.columns = ['user_carts_count','user_carts_item_count']

user_features3 = dataall[dataall.type=='orders'].groupby('session').agg({'session':'count','aid':'nunique'})
user_features3.columns = ['user_orders_count','user_orders_item_count']

user_features = pd.concat([user_features0,user_features1,user_features2,user_features3],axis=1)


In [21]:
user_features.to_parquet('user_features_test.pqt')

In [22]:
dataall0 = pd.concat([dataall,traindataall])
dataall0['type_number'] = dataall0['type'].apply(lambda x:type_weight_multipliers[x])
dataall0.shape

(223644219, 5)

In [23]:
item_features0 = dataall0.groupby('aid').agg({'aid':'count','session':'nunique','type_number':'mean'})
item_features0.columns = ['item_count','item_user_count','item_buy_ratio']
item_features0

,item_count,item_user_count,item_buy_ratio
aid,,,
0,48,40,1.000000
1,34,30,1.147059
2,17,16,1.000000
3,2759,1392,1.401232
4,221,143,1.203620
...,...,...,...
1855598,7,7,1.000000
1855599,14,12,1.000000
1855600,91,57,1.406593


In [24]:
item_features1 = dataall0[dataall0.type=='clicks'].groupby('aid').agg({'aid':'count','session':'nunique'})
item_features1.columns = ['item_clicks_count','item_clicks_user_count']

item_features2 = dataall0[dataall0.type=='carts'].groupby('aid').agg({'aid':'count','session':'nunique'})
item_features2.columns = ['item_carts_count','item_carts_user_count']

item_features3 = dataall0[dataall0.type=='orders'].groupby('aid').agg({'aid':'count','session':'nunique'})
item_features3.columns = ['item_orders_count','item_orders_user_count']


In [25]:
item_features = pd.concat([item_features0,item_features1,item_features2,item_features3],axis=1)
item_features

,item_count,item_user_count,item_buy_ratio,item_clicks_count,item_clicks_user_count,item_carts_count,item_carts_user_count,item_orders_count,item_orders_user_count
aid,,,,,,,,,
0,48,40,1.000000,48,40,NaN,NaN,NaN,NaN
1,34,30,1.147059,33,29,1.0,1.0,NaN,NaN
2,17,16,1.000000,17,16,NaN,NaN,NaN,NaN
3,2759,1392,1.401232,2513,1392,205.0,173.0,41.0,36.0
4,221,143,1.203620,212,143,9.0,9.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1855598,7,7,1.000000,7,7,NaN,NaN,NaN,NaN
1855599,14,12,1.000000,14,12,NaN,NaN,NaN,NaN
1855600,91,57,1.406593,83,57,7.0,7.0,1.0,1.0


In [26]:
item_features.to_parquet('item_features_test.pqt')

In [27]:
dataall0 = dataall.copy()
dataall0['type'] = dataall0['type'].apply(lambda x:type_weight_multipliers[x])
dataall0

,session,aid,ts,type
0,13399779,110716,1661885147895,1
1,13399779,738837,1661885164597,1
2,13399780,557072,1661885147958,1
3,13399780,770350,1661885172583,1
4,13399781,1062698,1661885147979,1
...,...,...,...,...
6928118,13599775,232605,1661959824002,1
6928119,13599776,812507,1661958067254,1
6928120,13599776,587309,1661958145310,1
6928121,13599777,1583223,1661958067311,1


In [28]:
user_item_features0 = dataall0.groupby(['session','aid']).agg({'type':'count'})
# user_item_features0 = user_item_features0.reset_index()
user_item_features0.columns = ['user_item_int_count']

user_item_features1 = dataall[dataall.type=='clicks'].groupby(['session','aid']).agg({'type':'count'})
# user_item_features1 = user_item_features1.reset_index()
user_item_features1.columns = ['user_item_int_clicks_count']

user_item_features2 = dataall[dataall.type=='carts'].groupby(['session','aid']).agg({'type':'count'})
# user_item_features2 = user_item_features2.reset_index()
user_item_features2.columns = ['user_item_int_carts_count']

user_item_features3 = dataall[dataall.type=='orders'].groupby(['session','aid']).agg({'type':'count'})
# user_item_features3 = user_item_features3.reset_index()
user_item_features3.columns = ['user_item_int_orders_count']

user_item_features = pd.concat([user_item_features0,user_item_features1,user_item_features2,user_item_features3],axis=1)


In [29]:
user_item_features.to_parquet('user_item_features_test.pqt')

In [9]:
import pickle
import gensim
ff = open('../w2vmodel/w2vec.pkl','rb')
w2vec = pickle.load(ff)
ff.close()

In [10]:
userall = dataall.groupby(['session','type'])['aid'].apply(list)
userall = userall.reset_index()

In [8]:
import numpy as np
from tqdm import tqdm
ww1 = {}
userall1 = userall[userall.type=='clicks'].reset_index(drop=True)
for ii in tqdm(range(len(userall1))):
    sss = [w2vec.wv[xx] for xx in userall1.loc[ii,'aid']]
    ww1[userall1.loc[ii,'session']] =np.mean(sss,axis=0)

100%|██████████| 1670191/1670191 [03:08<00:00, 8852.57it/s] 


In [9]:
ww2 = {}
userall2 = userall[userall.type=='carts'].reset_index(drop=True)
for ii in tqdm(range(len(userall2))):
    sss = [w2vec.wv[xx] for xx in userall2.loc[ii,'aid']]
    ww2[userall2.loc[ii,'session']] =np.mean(sss,axis=0)

100%|██████████| 242833/242833 [00:16<00:00, 14660.97it/s]


In [10]:
ww3 = {}
userall3 = userall[userall.type=='orders'].reset_index(drop=True)
for ii in tqdm(range(len(userall3))):
    sss = [w2vec.wv[xx] for xx in userall3.loc[ii,'aid']]
    ww3[userall3.loc[ii,'session']] =np.mean(sss,axis=0)

100%|██████████| 35849/35849 [00:02<00:00, 17702.92it/s]


In [ ]:
import pickle
ff = open(f'ww1clicks_test.pkl','wb')
pickle.dump(ww1,ff)
ff.close()

ff = open(f'ww2carts_test.pkl','wb')
pickle.dump(ww2,ff)
ff.close()

ff = open(f'ww3orders_test.pkl','wb')
pickle.dump(ww3,ff)
ff.close()

In [11]:
import numpy as np
from tqdm import tqdm
ww1 = {}
userall1 = userall[userall.type=='clicks'].reset_index(drop=True)
for ii in tqdm(range(len(userall1))):
    sss = [w2vec.wv[xx] for xx in userall1.loc[ii,'aid'][-1:]]
    ww1[userall1.loc[ii,'session']] =np.mean(sss,axis=0)
    
ww2 = {}
userall2 = userall[userall.type=='carts'].reset_index(drop=True)
for ii in tqdm(range(len(userall2))):
    sss = [w2vec.wv[xx] for xx in userall2.loc[ii,'aid'][-1:]]
    ww2[userall2.loc[ii,'session']] =np.mean(sss,axis=0)
    
ww3 = {}
userall3 = userall[userall.type=='orders'].reset_index(drop=True)
for ii in tqdm(range(len(userall3))):
    sss = [w2vec.wv[xx] for xx in userall3.loc[ii,'aid'][-1:]]
    ww3[userall3.loc[ii,'session']] =np.mean(sss,axis=0)
    
import pickle
ff = open(f'ww1clicks_last1_test.pkl','wb')
pickle.dump(ww1,ff)
ff.close()

ff = open(f'ww2carts_last1_test.pkl','wb')
pickle.dump(ww2,ff)
ff.close()


ff = open(f'ww3orders_last1_test.pkl','wb')
pickle.dump(ww3,ff)
ff.close()

100%|██████████| 35849/35849 [00:02<00:00, 17214.50it/s]


In [12]:
import numpy as np
from tqdm import tqdm
ww1 = {}
userall1 = userall[userall.type=='clicks'].reset_index(drop=True)
for ii in tqdm(range(len(userall1))):
    sss = [w2vec.wv[xx] for xx in userall1.loc[ii,'aid'][-2:]]
    ww1[userall1.loc[ii,'session']] =np.mean(sss,axis=0)
    
    
import pickle
ff = open(f'ww1clicks_last2_test.pkl','wb')
pickle.dump(ww1,ff)
ff.close()

100%|██████████| 1670191/1670191 [02:13<00:00, 12548.33it/s]


In [13]:
import numpy as np
from tqdm import tqdm
ww1 = {}
userall1 = userall[userall.type=='clicks'].reset_index(drop=True)
for ii in tqdm(range(len(userall1))):
    sss = [w2vec.wv[xx] for xx in userall1.loc[ii,'aid'][-3:]]
    ww1[userall1.loc[ii,'session']] =np.mean(sss,axis=0)
    
    
import pickle
ff = open(f'ww1clicks_last3_test.pkl','wb')
pickle.dump(ww1,ff)
ff.close()

100%|██████████| 1670191/1670191 [02:14<00:00, 12445.10it/s]


In [14]:
ww1 = {}
userall1 = userall[userall.type=='clicks'].reset_index(drop=True)
for ii in tqdm(range(len(userall1))):
    sss = [w2vec.wv[xx] for xx in userall1.loc[ii,'aid'][-5:]]
    ww1[userall1.loc[ii,'session']] =np.mean(sss,axis=0)
    
    
import pickle
ff = open(f'ww1clicks_last5_test.pkl','wb')
pickle.dump(ww1,ff)
ff.close()

100%|██████████| 1670191/1670191 [02:14<00:00, 12382.76it/s]


In [15]:
ww1 = {}
userall1 = userall[userall.type=='clicks'].reset_index(drop=True)
for ii in tqdm(range(len(userall1))):
    sss = [w2vec.wv[xx] for xx in userall1.loc[ii,'aid'][-10:]]
    ww1[userall1.loc[ii,'session']] =np.mean(sss,axis=0)
    
    
import pickle
ff = open(f'ww1clicks_last10_test.pkl','wb')
pickle.dump(ww1,ff)
ff.close()

100%|██████████| 1670191/1670191 [02:17<00:00, 12178.22it/s]
